In [2]:
import pandas as pd
import numpy as np

In [1]:
df_resale = pd.read_csv("../data/cleaned/ResalePrice_amenities_coor.csv")
df_resale

NameError: name 'pd' is not defined

## Adjust resale price based on inflation rate

In [ ]:
df_CPI = pd.read_csv("../data/cleaned/SG_Inflation_Data.csv")

df_resale = df_resale.merge(df_CPI, left_on="month", right_on="Time", how="left")
df_resale.drop(columns=["Time"], inplace=True)
df_resale['resale_price'] = df_resale['resale_price']*100/df_resale['CPI (base 2024-12)']
df_resale


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,nearest_mrt_distance,nearest_bus_distance,LatLng,education_score,shopping_score,food_score,recreation_score,healthcare_score,CPI (base 2024-12),inflation_rate (x100)
0,2017-03,BUKIT MERAH,3 ROOM,43,TELOK BLANGAH RISE,13 TO 15,59.0,Improved,1976,58 years 01 month,...,580.883827,100.652841,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,84.527761,0.041405
1,2017-05,BUKIT MERAH,3 ROOM,43,TELOK BLANGAH RISE,07 TO 09,59.0,Improved,1976,57 years 11 months,...,580.883827,100.652841,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,84.556772,0.340212
2,2018-01,BUKIT MERAH,3 ROOM,43,TELOK BLANGAH RISE,10 TO 12,59.0,Improved,1976,57 years 03 months,...,580.883827,100.652841,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,84.503016,-0.191484
3,2018-06,BUKIT MERAH,3 ROOM,43,TELOK BLANGAH RISE,01 TO 03,59.0,Improved,1976,56 years 11 months,...,580.883827,100.652841,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,84.904050,0.056310
4,2018-06,BUKIT MERAH,3 ROOM,43,TELOK BLANGAH RISE,04 TO 06,59.0,Improved,1976,56 years 11 months,...,580.883827,100.652841,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,84.904050,0.056310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201055,2024-07,SEMBAWANG,5 ROOM,484,ADMIRALTY LINK,07 TO 09,110.0,Improved,2003,78 years 01 month,...,1035.713889,117.518636,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,99.002534,-0.253604
201056,2024-08,SEMBAWANG,4 ROOM,484,ADMIRALTY LINK,13 TO 15,91.0,Model A,2003,78 years,...,1035.713889,117.518636,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,99.685999,0.690351
201057,2024-10,SEMBAWANG,5 ROOM,484,ADMIRALTY LINK,13 TO 15,111.0,Improved,2003,77 years 10 months,...,1035.713889,117.518636,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,99.623711,-0.313346
201058,2024-10,SEMBAWANG,5 ROOM,484,ADMIRALTY LINK,07 TO 09,110.0,Improved,2003,77 years 10 months,...,1035.713889,117.518636,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,99.623711,-0.313346


## Convert lease_commence_date into building age by 2025

In [ ]:
df_resale['building_age_2025'] = 2025 - df_resale['lease_commence_date']

## Convert to numeric 

In [ ]:
# Convert remaining_lease into numeric variables
def convert_to_years(age_str):
    parts = age_str.split()
    years = int(parts[0])
    months = 0
    if len(parts) > 2:
        months = int(parts[2])
    return round(years + (months / 12),2)

df_resale['remaining_lease'] = df_resale['remaining_lease'].apply(convert_to_years)

In [ ]:
flat_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'MULTI-GENERATION': 6,
    'EXECUTIVE': 7
}

df_resale['flat_type'] = df_resale['flat_type'].map(flat_mapping)
df_resale['storey_range'] = df_resale['storey_range'].str[:2].astype(int)


In [ ]:
df_resale = df_resale.drop(columns=['address','street_name', 'block', 'LatLng', 'lease_commence_date'])

## Add Macro variables

### Unemployment Rate

In [ ]:
# Choose relevant rows
df_unemployment = pd.read_csv("../data/raw/unemployment.csv")
df_unemployment = df_unemployment.iloc[9:12]
df_unemployment.columns = df_unemployment.iloc[0]

# Pivot the columns
df_unemployment = df_unemployment[1:]
df_unemployment = df_unemployment.melt(id_vars=["Data Series"], var_name="year", value_name="unemployment_rate")
df_unemployment["year"] = df_unemployment["year"].astype(int)
df_unemployment = df_unemployment.pivot(index="year", columns="Data Series", values="unemployment_rate").reset_index()
df_unemployment.columns = ["year", "total_unemployment_rate", "resident_unemployment_rate"]

# Join with resale dataframe
df_resale["year"] = df_resale["month"].str[:4].astype(int)
df_resale = df_resale.merge(df_unemployment, on="year", how="left")
df_resale = df_resale.drop(columns=['year'])

### Interest rate

Choose Singapore Overnight Rate Average (SORA) as the indicator for interest rate/

In [ ]:
df_interest = pd.read_csv("../data/raw/interest_rate.csv",skiprows=10)
df_interest = df_interest[:8]
df_interest = df_interest.iloc[[6]]
df_interest

,Data Series,2025 Jan,2024 Dec,2024 Nov,2024 Oct,2024 Sep,2024 Aug,2024 Jul,2024 Jun,2024 May,...,1988 Oct,1988 Sep,1988 Aug,1988 Jul,1988 Jun,1988 May,1988 Apr,1988 Mar,1988 Feb,1988 Jan
6,Singapore Overnight Rate Average,2.6325,2.1123,3.0839,3.1559,3.8808,3.5355,3.4336,3.4325,3.5222,...,na,na,na,na,na,na,na,na,na,na


In [ ]:
df_interest.replace('na', np.nan, inplace=True)
df_interest = df_interest.dropna(axis=1)

df_interest = df_interest.melt(var_name="month", value_name="interest_rate")
df_interest = df_interest[1:]

/var/folders/pf/xn094ngs70lb3jh0xpvlzx8c0000gn/T/ipykernel_85308/1638764280.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_interest.replace('na', np.nan, inplace=True)


In [ ]:
# Convert month to "YYYY-MM" date format
df_interest["month"] = df_interest["month"].str.strip()
df_interest["month"] = pd.to_datetime(df_interest["month"], format="%Y %b", errors="coerce")
df_interest = df_interest.dropna(subset=["month"])
df_interest["month"] = df_interest["month"].dt.strftime("%Y-%m")

# Join with resale dataframe
df_resale = df_resale.merge(df_interest, on="month", how="left")
df_resale

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,lat,lon,...,shopping_score,food_score,recreation_score,healthcare_score,CPI (base 2024-12),inflation_rate (x100),building_age_2025,total_unemployment_rate,resident_unemployment_rate,interest_rate
0,2017-03,BUKIT MERAH,3,13,59.0,Improved,58.08,343082.552693,1.270380,103.823236,...,31.231448,14.302518,29.626696,55.974066,84.527761,0.041405,49,3.1,2.2,0.5687
1,2017-05,BUKIT MERAH,3,7,59.0,Improved,57.92,342964.842883,1.270380,103.823236,...,31.231448,14.302518,29.626696,55.974066,84.556772,0.340212,49,3.1,2.2,0.4503
2,2018-01,BUKIT MERAH,3,10,59.0,Improved,57.25,325432.170445,1.270380,103.823236,...,31.231448,14.302518,29.626696,55.974066,84.503016,-0.191484,49,2.9,2.1,0.8294
3,2018-06,BUKIT MERAH,3,1,59.0,Improved,56.92,294450.027637,1.270380,103.823236,...,31.231448,14.302518,29.626696,55.974066,84.904050,0.056310,49,2.9,2.1,1.3426
4,2018-06,BUKIT MERAH,3,4,59.0,Improved,56.92,300339.028190,1.270380,103.823236,...,31.231448,14.302518,29.626696,55.974066,84.904050,0.056310,49,2.9,2.1,1.3426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201055,2024-07,SEMBAWANG,5,7,110.0,Improved,78.08,555541.334850,1.457071,103.815308,...,30.596328,1.233127,25.048367,8.851718,99.002534,-0.253604,22,2.7,2.0,3.4336
201056,2024-08,SEMBAWANG,4,13,91.0,Model A,78.00,501574.951425,1.457071,103.815308,...,30.596328,1.233127,25.048367,8.851718,99.685999,0.690351,22,2.7,2.0,3.5355
201057,2024-10,SEMBAWANG,5,13,111.0,Improved,77.83,630372.023708,1.457071,103.815308,...,30.596328,1.233127,25.048367,8.851718,99.623711,-0.313346,22,2.7,2.0,3.1559
201058,2024-10,SEMBAWANG,5,7,110.0,Improved,77.83,552077.409298,1.457071,103.815308,...,30.596328,1.233127,25.048367,8.851718,99.623711,-0.313346,22,2.7,2.0,3.1559


### Foreign Exchange Rate

In [ ]:
df_fxrate = pd.read_csv("../data/raw/fxrate.csv", skiprows=9).head(1)

df_fxrate.replace('na', np.nan, inplace=True)
df_fxrate = df_fxrate.dropna(axis=1)

df_fxrate = df_fxrate.melt(var_name="month", value_name="fx_rate")[1:]
df_fxrate

,month,fx_rate
1,2025 Jan,1.3625
2,2024 Dec,1.3503
3,2024 Nov,1.3365
4,2024 Oct,1.309
5,2024 Sep,1.2964
...,...,...
441,1988 May,2.0098
442,1988 Apr,2.0037
443,1988 Mar,2.0133
444,1988 Feb,2.0173


In [ ]:
# Convert month to "YYYY-MM" date format
df_fxrate["month"] = df_fxrate["month"].str.strip()
df_fxrate["month"] = pd.to_datetime(df_fxrate["month"], format="%Y %b", errors="coerce")
df_fxrate = df_fxrate.dropna(subset=["month"])
df_fxrate["month"] = df_fxrate["month"].dt.strftime("%Y-%m")

# Join with resale dataframe
df_resale = df_resale.merge(df_fxrate, on="month", how="left")
df_resale

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,lat,lon,...,food_score,recreation_score,healthcare_score,CPI (base 2024-12),inflation_rate (x100),building_age_2025,total_unemployment_rate,resident_unemployment_rate,interest_rate,fx_rate
0,2017-03,BUKIT MERAH,3,13,59.0,Improved,58.08,343082.552693,1.270380,103.823236,...,14.302518,29.626696,55.974066,84.527761,0.041405,49,3.1,2.2,0.5687,1.4057
1,2017-05,BUKIT MERAH,3,7,59.0,Improved,57.92,342964.842883,1.270380,103.823236,...,14.302518,29.626696,55.974066,84.556772,0.340212,49,3.1,2.2,0.4503,1.3946
2,2018-01,BUKIT MERAH,3,10,59.0,Improved,57.25,325432.170445,1.270380,103.823236,...,14.302518,29.626696,55.974066,84.503016,-0.191484,49,2.9,2.1,0.8294,1.322
3,2018-06,BUKIT MERAH,3,1,59.0,Improved,56.92,294450.027637,1.270380,103.823236,...,14.302518,29.626696,55.974066,84.904050,0.056310,49,2.9,2.1,1.3426,1.3474
4,2018-06,BUKIT MERAH,3,4,59.0,Improved,56.92,300339.028190,1.270380,103.823236,...,14.302518,29.626696,55.974066,84.904050,0.056310,49,2.9,2.1,1.3426,1.3474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201055,2024-07,SEMBAWANG,5,7,110.0,Improved,78.08,555541.334850,1.457071,103.815308,...,1.233127,25.048367,8.851718,99.002534,-0.253604,22,2.7,2.0,3.4336,1.3469
201056,2024-08,SEMBAWANG,4,13,91.0,Model A,78.00,501574.951425,1.457071,103.815308,...,1.233127,25.048367,8.851718,99.685999,0.690351,22,2.7,2.0,3.5355,1.3159
201057,2024-10,SEMBAWANG,5,13,111.0,Improved,77.83,630372.023708,1.457071,103.815308,...,1.233127,25.048367,8.851718,99.623711,-0.313346,22,2.7,2.0,3.1559,1.309
201058,2024-10,SEMBAWANG,5,7,110.0,Improved,77.83,552077.409298,1.457071,103.815308,...,1.233127,25.048367,8.851718,99.623711,-0.313346,22,2.7,2.0,3.1559,1.309


## Normalise continuous variables

In [ ]:
df_resale_normalized = df_resale
def normalize(col):
    return (col - col.min()) / (col.max() - col.min()) if col.max() != col.min() else col

columns_to_normalize = df_resale.select_dtypes(include=['float64', 'int64']).columns
df_resale_normalized[columns_to_normalize] = df_resale[columns_to_normalize].apply(normalize)
df_resale_normalized


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,lat,lon,...,food_score,recreation_score,healthcare_score,CPI (base 2024-12),inflation_rate (x100),building_age_2025,total_unemployment_rate,resident_unemployment_rate,interest_rate,fx_rate
0,2017-03,BUKIT MERAH,0.333333,0.2500,0.083408,Improved,0.312121,0.131278,0.0,0.456109,...,0.290546,0.112261,0.169271,0.018299,0.444476,0.818182,3.1,2.2,0.5687,1.4057
1,2017-05,BUKIT MERAH,0.333333,0.1250,0.083408,Improved,0.309346,0.131197,0.0,0.456109,...,0.290546,0.112261,0.169271,0.020140,0.582388,0.818182,3.1,2.2,0.4503,1.3946
2,2018-01,BUKIT MERAH,0.333333,0.1875,0.083408,Improved,0.297728,0.119126,0.0,0.456109,...,0.290546,0.112261,0.169271,0.016729,0.336988,0.818182,2.9,2.1,0.8294,1.322
3,2018-06,BUKIT MERAH,0.333333,0.0000,0.083408,Improved,0.292006,0.097795,0.0,0.456109,...,0.290546,0.112261,0.169271,0.042174,0.451355,0.818182,2.9,2.1,1.3426,1.3474
4,2018-06,BUKIT MERAH,0.333333,0.0625,0.083408,Improved,0.292006,0.101849,0.0,0.456109,...,0.290546,0.112261,0.169271,0.042174,0.451355,0.818182,2.9,2.1,1.3426,1.3474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201055,2024-07,SEMBAWANG,0.666667,0.1250,0.235329,Improved,0.658921,0.277557,1.0,0.429908,...,0.018395,0.083381,0.026768,0.936712,0.308317,0.327273,2.7,2.0,3.4336,1.3469
201056,2024-08,SEMBAWANG,0.500000,0.2500,0.178731,Model A,0.657534,0.240401,1.0,0.429908,...,0.018395,0.083381,0.026768,0.980077,0.743992,0.327273,2.7,2.0,3.5355,1.3159
201057,2024-10,SEMBAWANG,0.666667,0.2500,0.238308,Improved,0.654586,0.329079,1.0,0.429908,...,0.018395,0.083381,0.026768,0.976125,0.280743,0.327273,2.7,2.0,3.1559,1.309
201058,2024-10,SEMBAWANG,0.666667,0.1250,0.235329,Improved,0.654586,0.275172,1.0,0.429908,...,0.018395,0.083381,0.026768,0.976125,0.280743,0.327273,2.7,2.0,3.1559,1.309


In [ ]:
df_resale.to_csv('../data/cleaned/resale_price_cleaned.csv', index=False)
df_resale.to_csv('../data/cleaned/resale_price_normalized.csv', index=False)